# MLP Batch Normalization

A batch normalization implementation with manually calculated gradients.

In [1]:
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 1. Define Model

In [4]:
def forward_prop(X, params, num_layer):
    neurons = X
    m = X.shape[0]
    for i in range(num_layer):
        W = params['w' + str(i)]
        bn_scalar = params['beta' + str(i)]
        bn_bias = params['bias' + str(i)]
        linear_values = X @ W
        mean = np.mean(linear_values, axis=1)/m
        std = np.std(linear_values, axis=1)/m
        normalized_linear_values = (linear_values - mean)/(std + 1e-8)
        if i < num_layer - 1:
            activation_values = np.max(normalized_linear_values, 0)
        else:
            e_power = np.exp(normalized_linear_values)
            e_power_sum = np.sum(e_power)
            activation_values = e_power/e_power_sum
        neurons = activation_values
    return neurons